In [120]:
# imports
import seaborn as sns
import numpy as np
import pandas as pd
import json
from matplotlib import pyplot as plt
from scipy.stats import norm
import statistics
from cryptocmd import CmcScraper
import time
from sklearn.linear_model import LinearRegression
from datetime import datetime

# get data from json file
rawDf = pd.read_json(r'transactions-30days.json')

# define helper functions
def getReinvestReward(transaction):
    # find "from" in transaction
    value = 0
    counter = 0
    index = ""
    for x in transaction:
        if x == transaction["from"]:
            if counter == 1:
                index = transaction.index[transaction==x].tolist()
                #print(index)
                newIndex = str(index[1][:-2]) + "for"
                value = transaction[str(newIndex)]
                break
            counter = counter + 1
    return value

def getReinvestedValue(transaction):
    # find "from" in transaction
    value = 0
    counter = 0
    index = ""
    for x in transaction:
        if x == transaction["interacted_with"]:
            if counter == 1:
                index = transaction.index[transaction==x].tolist()
                #print(index)
                newIndex = str(index[1][:-2]) + "for"
                value = transaction[str(newIndex)]
                break
            counter = counter + 1
    return value

In [66]:
# contracts and their interaction count
contracts = rawDf["interacted_with"].unique()
rawDf["interacted_with"].value_counts()

# remove nan from contracts
contracts2 = []
for i in contracts:
    if type(i) != float:
        contracts2.append(i)

# read in contracts and reward currencies
input_file = open ('contracts_and_reward_currency.json')
contracts_and_reward_currency = json.load(input_file)

# split contract addresses and reward currencies
ll = []
ll2 = []
for i in range(len(contracts_and_reward_currency)):
    if (i % 2 == 0):
        ll.append(contracts_and_reward_currency[i])
    if (i % 2 != 0):
        ll2.append(contracts_and_reward_currency[i])

# create 2d array of contract addresses and reward currencies
_2d_contracts_and_reward_currencies = []
for x in range(len(ll)):
    _2d_contracts_and_reward_currencies.append([ll[x], ll2[x]])
    
# get contracts with avax as reward
contracts_with_avax_reward = []
for pair in _2d_contracts_and_reward_currencies:
    if pair[1] == "AVAX":
        contracts_with_avax_reward.append(pair[0])

# get transactions related to contracts with avax as reward
transactions_with_avax_reward = []
for i in range(len(rawDf)):
    if rawDf.iloc[i]["interacted_with"] in contracts_with_avax_reward:
        transactions_with_avax_reward.append(rawDf.iloc[i])

In [53]:
# get all unique reward currencies
reward_currencies = np.unique(ll2)

# get historical data for reward currencies
reward_currencies_historical = []
for symbol in reward_currencies:
    if symbol != "SHIT":
        scraper = CmcScraper(symbol, "01-03-2022", "01-04-2022")
        reward_currencies_historical.append([symbol, scraper.get_dataframe()])

[['AVAX',
           Date       Open        High        Low      Close        Volume  \
  0  2022-04-01  97.121146   99.653044  90.777600  96.693181  1.744656e+09   
  1  2022-03-31  95.692750  101.685378  94.261105  96.924663  2.201716e+09   
  2  2022-03-30  92.813955  100.078888  90.002954  95.779984  1.755846e+09   
  3  2022-03-29  89.523987   95.876268  89.514575  92.840892  1.642342e+09   
  4  2022-03-28  89.842906   96.075850  88.663393  89.604551  1.758255e+09   
  5  2022-03-27  85.956847   89.824848  84.566085  89.824848  7.590681e+08   
  6  2022-03-26  83.670632   86.689308  82.611274  85.955653  6.663118e+08   
  7  2022-03-25  87.548559   87.629725  82.808252  83.661369  1.128009e+09   
  8  2022-03-24  86.230186   88.105563  83.864928  87.555386  1.626543e+09   
  9  2022-03-23  83.998290   86.891878  82.522202  86.251013  1.478084e+09   
  10 2022-03-22  86.418175   89.703661  83.877186  83.981448  1.746086e+09   
  11 2022-03-21  84.708328   92.308269  83.444131  86.

In [136]:
linear_regressor = linear_model.LinearRegression()

print(pd.to_datetime(reward_currencies_historical[0][1]["Date"][0] - pd.Timestamp("1970-01-01")))

X = reward_currencies_historical[0][1]["Close"].values.reshape(-1, 1)
Y = reward_currencies_historical[0][1]["Date"].values.reshape(-1, 1)

#linear_regressor.fit(X, Y)
#Y_pred = linear_regressor.predict(X)

# plot it as in the example at http://scikit-learn.org/
#plt.scatter(X, Y)
#plt.plot(X, Y_pred, color='red')
#plt.show()

TypeError: <class 'pandas._libs.tslibs.timedeltas.Timedelta'> is not convertible to datetime

In [19]:
# profitability per contract
# get reinvest reward percentage
arr = []
for contract in contracts2:
    for i in range(len(transactions_with_avax_reward)):
        elem = transactions_with_avax_reward[i]
        if elem["interacted_with"] == contract:
            try:
                temp = [elem["from"], getReinvestReward(elem), getReinvestedValue(elem), elem["transaction_fee"]]
                arr.append(temp)
            except:
                print("yosha")

# filter out shit values
rr = []
for r in arr:
    if r[2] < 1000:
        rr.append(r)

# calc profitablilty percentage
cc = []
for b in rr:
    if float(b[2]) != 0:
        cc.append([b[0], b[1], b[2], b[3], float(b[1])/float(b[2])*100])
    else:
        cc.append([b[0], b[1], b[2], b[3], "err: div by 0"])

# total profitablility
# calc profitablilty percentage
dd = []
for c in cc:
        dd.append([c[0], c[1], c[2], c[3], c[4], float(c[1])-float(c[3])])

# avg
sum = 0
for d in dd:
    sum += d[5]

print(float(sum)/float(len(dd)))

yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
yosha
-0.003925984889391453


In [31]:
#convert to dataframe
df = pd.DataFrame(dd, 
             columns=['from', 'reinvest_reward',
                      'total_reinvested', 'transaction_fee',
                      'total_to_reward_ratio', 'total_profit'])
df

,from,reinvest_reward,total_reinvested,transaction_fee,total_to_reward_ratio,total_profit
0,0x4a4edacd5fcecf34c457f4198ddd5385de0dbe85,0.013362,13.362382,0.021450,0.1,-0.008087
1,0x2ec0a34de9e2f9577e20b39fbb9f85106cf2e768,0.000029,0.029041,0.023789,0.1,-0.023760
2,0xdaa99a4826b3d0e9998cb55c8c648c53a5be677e,0.027810,27.810380,0.024214,0.1,0.003596
3,0x46c2da32fc5effdbc1d328ba873c925568e41564,0.010573,10.573113,0.029400,0.1,-0.018826
4,0x4a9a40b2c44b7d32ecf4e0cc66f5304518ec0a59,0.000031,0.031051,0.035520,0.1,-0.035488
...,...,...,...,...,...,...
6313,0x0783abc1e6f1eb93b80024a056492e314d949986,0.002960,5.611620,0.121274,0.052741,-0.118315
6314,0x0783abc1e6f1eb93b80024a056492e314d949986,0.004275,13.810728,0.048140,0.030954,-0.043865
6315,0x0783abc1e6f1eb93b80024a056492e314d949986,0.003508,4.915227,0.118199,0.07137,-0.114691
6316,0x0783abc1e6f1eb93b80024a056492e314d949986,0.008325,62.301689,0.049900,0.013362,-0.041575


In [37]:
_77e = df.loc[df['from'] == "0xdaa99a4826b3d0e9998cb55c8c648c53a5be677e"]
_77e.sum()

C:\Users\David\AppData\Local\Temp\ipykernel_21048\710772087.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  _77e.sum()


from                0xdaa99a4826b3d0e9998cb55c8c648c53a5be677e0xda...
reinvest_reward                                            101.031371
total_reinvested                                        500644.589865
transaction_fee                                             93.976666
total_profit                                                 7.054705
dtype: object

In [40]:
df1 = df.groupby("from").sum().sort_values("total_profit")
df2 = df1.loc[df1['total_profit'] > 0]
df2.sum()

reinvest_reward        166.872263
total_reinvested    642676.886178
transaction_fee        138.524648
total_profit            28.347615
dtype: float64

In [22]:
df.groupby("from").mean().sort_values("total_profit")

,reinvest_reward,total_reinvested,transaction_fee,total_profit
from,,,,
0x66001fb5f11abc3a63fa276538fc76102c11f3e5,0.001608,0.357199,0.320479,-0.318871
0xcbf8cdb444ee8c5b67f007f9a46d7f16a3bde40a,0.037617,4.182778,0.282794,-0.245177
0x4e14780cad0e646502985f2c2cd9f688a50d649c,0.001109,6.103331,0.186184,-0.185075
0xfba9d51940d25eca570d59916a315481a2d89955,0.002522,75.273693,0.169387,-0.166865
0x14fca733f9283caaa6d0f2294239ff379b29fd0a,0.008026,10.874330,0.155342,-0.147317
...,...,...,...,...
0x0c642e90b4ae7fbf1706d635401e05692e63f681,0.058108,98.154277,0.037176,0.020932
0x56cfe5a4bd2dae4c294817b2bd059f90b037384d,0.046993,3.599291,0.023540,0.023454
0x000010ed59bcb8fac3aecb568aefbf8b49daea02,0.071954,399.809583,0.043859,0.028095
